In [1]:
%matplotlib
#%matplotlib inline
#import mpld3
#mpld3.enable_notebook()

from numpy import array, identity, zeros, stack
from numpy.random import randn
from numpy.linalg import cholesky
from bokeh.plotting import figure, gridplot, show
from kalman_lqg import kalman_lqg
from iLQG import iterative_lqg, compute_trajectories, compute_cost

# define constants
nx = 2
nu = 2
nw = 2
ny = 2
nv = 2
A = array([[1, 2], [3, 4]])
B = 1.0/A
C0 = cholesky(array([[0.11, 0],[0, 0.11]]))
H = array([[5, 3], [2, 1]])
D0 = cholesky(array([[0.1, 0],[0, 0.1]]))
Q = array([[4, 1],[1, 3]])
R = array([[0.3, 0.1],[0.1, 0.4]])
#R = zeros([2,2])
Qf = array([[4.1, 1],[1, 3.2]])


def f(x, u):
    """ If x(k+1) = Ax(k) + Bu(k) then f(x,u) = (A-I)x(k) + Bu(k) """
    I = identity(nx)
    return (A-I).dot(x) + B.dot(u) 

def F(x, u):
    return C0

def g(x, u):
    return H.dot(x)

def G(x, u):
    return D0

def h(x):
    return x.dot(Qf).dot(x)

def l(x, u):
    return x.dot(Q).dot(x) + u.dot(R).dot(u)
    


# find the optimal solution for this LQG system using kalman_lqg.py
N = 4 # number of time steps + 1
x0 = array([100,100])
system = {}
system['X1'] = x0
system['S1'] = identity(nx)
system['A'] = A
system['B'] = B
system['C0'] = C0
system['C'] = zeros([nu, nu, 1])
system['H'] = H
system['D0'] = D0
system['D'] = zeros([ny, nx, 1])
system['E0'] = zeros([nx, 1])
system['Q'] = stack([Q if k < N-1 else Qf for k in range(N)], -1)
system['R'] = R
 
#######################################################################
# This code was used to make sure MATLAB code produces the same results
#######################################################################
#import matlab
#import matlab.engine
#from test_kalman_lqg import matlab_kalman_lqg
#eng = matlab.engine.start_matlab()
#matlab_returns = matlab_kalman_lqg(eng, system)
# parse MATLAB return values according to the function definition
#function [K,L,Cost,Xa,XSim,CostSim,iter] = ...
#K = array(matlab_returns[0])
#L = array(matlab_returns[1])
#Cost = array(matlab_returns[2])
#Xa = array(matlab_returns[3])
#XSim = array(matlab_returns[4])
#CostSim = array(matlab_returns[5])
# I added this return value to make sure the algorithm converged
#iterations = array(matlab_returns[6])
#eng.quit()

K, L, Cost, Xa, XSim, CostSim, iterations = kalman_lqg(system)
x_lqg, u_lqg = compute_trajectories(f, x0, zeros([nu,N-1]), -L)

# iLQG should produce the same solution
x_ilqg, u_ilqg, Lx, lx = iterative_lqg(f, F, g, G, h, l, x0, N)

# print the cost for each algorithm's trajectories
print
print "kalman_lqg cost:"
print compute_cost(f, h, l, x_lqg, u_lqg)
print
print "iLQG cost:"
print compute_cost(f, h, l, x_ilqg, u_ilqg)

# plot the state trajectories
kx = range(N)
p1 = figure(title="State Trajectories", x_axis_label='time', y_axis_label='')
p1.line(kx, x_lqg[0,:], line_width=2, line_color="blue", legend="LQG x[0]")
p1.line(kx, x_lqg[1,:], line_width=2, line_color="green", legend="LQG x[1]")
p1.line(kx, x_ilqg[0,:], line_width=2, line_color="blue", line_dash='dashed',
        legend="iLQG x[0]")
p1.line(kx, x_ilqg[1,:], line_width=2, line_color="green", line_dash='dashed',
        legend="iLQG x[1]")
p1.legend.location = "bottom_right"

# plot the control trajectories
ku = range(N-1)
p2 = figure(title="Control Trajectories", x_axis_label='time', y_axis_label='')
p2.line(ku, u_lqg[0,:], line_width=2, line_color="blue", legend="LQG u[0]")
p2.line(ku, u_lqg[1,:], line_width=2, line_color="green", legend="LQG u[1]")
p2.line(ku, u_ilqg[0,:], line_width=2, line_color="blue", line_dash='dashed',
        legend="iLQG u[0]")
p2.line(ku, u_ilqg[1,:], line_width=2, line_color="green", line_dash='dashed',
        legend="iLQG u[1]")
p2.legend.location = "bottom_right"

p = gridplot([[p1], [p2]])
show(p)

Using matplotlib backend: MacOSX


TypeError: iterative_lqg() takes exactly 10 arguments (8 given)